# Bloco 1: Instalações, Importações e Função de Leitura

In [ ]:
# @title 1. Upload dos arquivos da instância e carregamento dos dados
!pip install pulp pandas matplotlib numpy -q

import os
import time
import json
import random
import pulp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
from google.colab import files
import shutil

# Variável global para acumular resultados de várias instâncias
if 'todos_resultados' not in globals():
    todos_resultados = []

def upload_instance_files():
    """Pede o nome da instância, cria a pasta e faz o upload dos arquivos."""
    inst_name = input("Digite o nome da instância (ex: i01, i04, i06): ").strip()
    if not os.path.exists(inst_name):
        os.makedirs(inst_name)

    print(f"\nSelecione os 3 arquivos da instância '{inst_name}':")
    print("(instance_info.json, nurse_shifts.csv, occupied_room_shifts.csv)")
    uploaded = files.upload()

    for filename in uploaded.keys():
        destino = os.path.join(inst_name, filename)
        if os.path.exists(destino):
            os.remove(destino)
        shutil.move(filename, destino)

    print(f"\nArquivos salvos em '{inst_name}'.")
    return inst_name

def load_instance(folder_path):
    """Lê os dados da instância a partir de uma pasta."""
    with open(os.path.join(folder_path, 'instance_info.json')) as f:
        info = json.load(f)

    # Extrai pesos (chaves conforme arquivo real)
    weights = {
        'skill': info['weights']['S2_room_nurse_skill'],
        'work': info['weights']['S4_nurse_excessive_workload']
    }
    df_nurses = pd.read_csv(os.path.join(folder_path, 'nurse_shifts.csv'))
    df_rooms = pd.read_csv(os.path.join(folder_path, 'occupied_room_shifts.csv'))
    return df_nurses, df_rooms, weights

# Executa upload e guarda o nome da instância atual
instancia_atual = upload_instance_files()
print("\nPronto! Instância carregada.")

# Bloco 2: O Resolvedor PLI (Exato):

In [ ]:
# @title 2. Resolvedor PLI (exato) - Versão corrigida
def solve_pli(df_nurses, df_rooms, weights, time_limit=None):
    """
    Resolve o NRA via PLI (PuLP).
    Retorna status, custo ótimo e tempo.
    time_limit: limite em segundos (opcional).
    """
    prob = pulp.LpProblem("NRA_Hospital", pulp.LpMinimize)

    # 1. Variáveis binárias: x[nurse_id, task_id]
    x = {}
    for idx, task in df_rooms.iterrows():
        task_id = idx
        g_shift = task['global_shift']
        nurses_avail = df_nurses[df_nurses['global_shift'] == g_shift]['nurse_id'].tolist()
        for n_id in nurses_avail:
            x[(n_id, task_id)] = pulp.LpVariable(f"x_{n_id}_{task_id}", cat='Binary')

    # 2. Restrição de cobertura: uma tarefa, um enfermeiro
    for idx, task in df_rooms.iterrows():
        task_id = idx
        g_shift = task['global_shift']
        nurses_avail = df_nurses[df_nurses['global_shift'] == g_shift]['nurse_id'].tolist()
        prob += pulp.lpSum(x[(n_id, task_id)] for n_id in nurses_avail if (n_id, task_id) in x) == 1, f"Cob_{task_id}"

    # 3. Variáveis para excesso de trabalho (inteiras)
    excess = {}
    for g_shift in df_rooms['global_shift'].unique():
        nurses_in_shift = df_nurses[df_nurses['global_shift'] == g_shift]['nurse_id'].tolist()
        for n_id in nurses_in_shift:
            excess[(n_id, g_shift)] = pulp.LpVariable(f"excess_{n_id}_{g_shift}", lowBound=0, cat='Integer')

    # 4. Função objetivo e restrições de excesso
    obj_terms = []

    # Penalidade de habilidade
    for idx, task in df_rooms.iterrows():
        task_id = idx
        req = task['max_skill_required']
        g_shift = task['global_shift']
        nurses_avail = df_nurses[df_nurses['global_shift'] == g_shift]
        for _, nurse in nurses_avail.iterrows():
            n_id = nurse['nurse_id']
            deficit = max(0, req - nurse['skill_level'])
            if (n_id, task_id) in x:
                obj_terms.append(weights['skill'] * deficit * x[(n_id, task_id)])

    # Penalidade de excesso de trabalho e restrições
    for g_shift in df_rooms['global_shift'].unique():
        tasks_shift = df_rooms[df_rooms['global_shift'] == g_shift]
        nurses_shift = df_nurses[df_nurses['global_shift'] == g_shift]
        for _, nurse in nurses_shift.iterrows():
            n_id = nurse['nurse_id']
            max_cap = nurse['max_load']
            e = excess[(n_id, g_shift)]

            total_work = pulp.lpSum(
                task['total_room_workload'] * x[(n_id, i)]
                for i, task in tasks_shift.iterrows()
                if (n_id, i) in x
            )
            prob += e >= total_work - max_cap, f"ExcessDef_{n_id}_{g_shift}"
            obj_terms.append(weights['work'] * e)

    prob += pulp.lpSum(obj_terms)

    # Resolução
    inicio = time.time()
    if time_limit:
        prob.solve(pulp.PULP_CBC_CMD(msg=False, timeLimit=time_limit))
    else:
        prob.solve(pulp.PULP_CBC_CMD(msg=False))
    fim = time.time()

    status = pulp.LpStatus[prob.status]
    custo = pulp.value(prob.objective) if prob.status == 1 else None
    tempo = fim - inicio
    return status, custo, tempo

print("Função PLI carregada (versão corrigida).")

# Bloco 3: Algoritmo Genético (Metaheurística):

In [ ]:
# @title 3. Algoritmo Genético Otimizado
def preprocess_nurses(df_nurses):
    """Cria dicionários para acesso rápido: (nurse_id, global_shift) -> (skill, max_load)"""
    nurse_info = {}
    for _, row in df_nurses.iterrows():
        nurse_info[(row['nurse_id'], row['global_shift'])] = (row['skill_level'], row['max_load'])
    return nurse_info

def calculate_fitness(solution, df_rooms, weights, nurse_info):
    cost = 0
    workload = {}

    for idx, n_id in enumerate(solution):
        task = df_rooms.iloc[idx]
        g_shift = task['global_shift']
        skill, _ = nurse_info[(n_id, g_shift)]  # (skill, max_load) - ignoramos max_load aqui
        deficit = max(0, task['max_skill_required'] - skill)
        cost += weights['skill'] * deficit

        key = (n_id, g_shift)
        workload[key] = workload.get(key, 0) + task['total_room_workload']

    for (n_id, g_shift), total in workload.items():
        _, max_cap = nurse_info[(n_id, g_shift)]
        if total > max_cap:
            cost += weights['work'] * (total - max_cap)

    return cost

def generate_individual(df_rooms, nurses_by_shift):
    """Gera um indivíduo aleatório factível."""
    ind = []
    for _, task in df_rooms.iterrows():
        g_shift = task['global_shift']
        available = nurses_by_shift[g_shift]
        ind.append(random.choice(available))
    return ind

def repair_individual(individual, df_rooms, nurses_by_shift):
    """Garante que cada posição tenha um enfermeiro disponível para o turno."""
    repaired = individual[:]
    for i, task in df_rooms.iterrows():
        g_shift = task['global_shift']
        available = nurses_by_shift[g_shift]
        if repaired[i] not in available:
            repaired[i] = random.choice(available)
    return repaired

def run_ga(df_nurses, df_rooms, weights, pop_size=50, generations=100, mutation_rate=0.1, repetitions=5):
    # Pré-processamento
    nurse_info = preprocess_nurses(df_nurses)
    nurses_by_shift = {}
    for g_shift in df_rooms['global_shift'].unique():
        nurses_by_shift[g_shift] = df_nurses[df_nurses['global_shift'] == g_shift]['nurse_id'].tolist()

    histories = []
    best_costs = []
    times = []

    for rep in range(repetitions):
        start = time.time()
        # Inicialização
        pop = [generate_individual(df_rooms, nurses_by_shift) for _ in range(pop_size)]
        history = []

        for gen in range(generations):
            # Avaliação
            fitness = [calculate_fitness(ind, df_rooms, weights, nurse_info) for ind in pop]
            sorted_idx = np.argsort(fitness)
            pop = [pop[i] for i in sorted_idx]
            fitness = [fitness[i] for i in sorted_idx]

            history.append(fitness[0])

            # Elitismo: mantém os 2 melhores
            new_pop = pop[:2]

            # Gera o restante da população
            while len(new_pop) < pop_size:
                # Seleciona dois pais dentre os 20 melhores
                candidates = random.sample(range(min(20, pop_size)), 2)
                p1 = pop[candidates[0]]
                p2 = pop[candidates[1]]

                # Crossover de um ponto
                point = random.randint(1, len(p1)-1)
                child = p1[:point] + p2[point:]

                # Reparo para garantir factibilidade
                child = repair_individual(child, df_rooms, nurses_by_shift)

                # Mutação adicional
                if random.random() < mutation_rate:
                    i = random.randint(0, len(child)-1)
                    g_shift = df_rooms.iloc[i]['global_shift']
                    child[i] = random.choice(nurses_by_shift[g_shift])

                new_pop.append(child)

            pop = new_pop

        end = time.time()
        histories.append(history)
        best_costs.append(min(fitness))  # melhor da última geração
        times.append(end - start)

    return histories, best_costs, times

print("Funções do GA otimizadas carregadas.")

# Bloco 4: Execução, Gráficos e Tabela:

In [ ]:
# @title 4. Executar PLI + GA e gerar tabela
print(f"\n{'='*50}")
print(f"Processando instância: {instancia_atual}")
print('='*50)

# Carregar dados
df_nurses, df_rooms, weights = load_instance(instancia_atual)
print(f"  Tarefas: {len(df_rooms)} | Enfermeiros: {len(df_nurses)}")

# --- PLI ---
print("\n [PLI] Buscando solução ótima... (limite de 300s)")
status, pli_cost, pli_time = solve_pli(df_nurses, df_rooms, weights, time_limit=300)

if status == 'Optimal':
    print(f"  Status: {status} | Custo: {pli_cost:.2f} | Tempo: {pli_time:.4f} s")
else:
    print(f"  Status: {status} (sem solução ótima)")
    pli_cost = None

# --- GA ---
print("\n [GA] Executando 5 repetições...")
# Para teste rápido, use parâmetros reduzidos (descomente a linha abaixo se quiser testar)
# histories, ga_costs, ga_times = run_ga(df_nurses, df_rooms, weights, pop_size=20, generations=20, repetitions=2)
histories, ga_costs, ga_times = run_ga(
    df_nurses, df_rooms, weights,
    pop_size=50, generations=100, mutation_rate=0.1, repetitions=5
)

ga_best = min(ga_costs)
ga_mean = np.mean(ga_costs)
ga_std = np.std(ga_costs)
ga_time_mean = np.mean(ga_times)
ga_time_std = np.std(ga_times)

print(f"  Melhor custo: {ga_best:.2f}")
print(f"  Custo médio: {ga_mean:.2f} ± {ga_std:.2f}")
print(f"  Tempo médio: {ga_time_mean:.4f} ± {ga_time_std:.4f} s")

# --- Gráfico de convergência ---
plt.figure(figsize=(8, 4))
for i, hist in enumerate(histories):
    plt.plot(hist, label=f'Exec {i+1}', alpha=0.7)
if pli_cost is not None and pli_cost > 0:
    plt.axhline(y=pli_cost, color='r', linestyle='--', label='Ótimo (PLI)')
plt.xlabel('Gerações')
plt.ylabel('Custo')
plt.title(f'Convergência do GA - Instância {instancia_atual}')
plt.legend()
plt.grid(True)
plt.tight_layout()
nome_grafico = f'convergencia_{instancia_atual}.png'
plt.savefig(nome_grafico, dpi=150)
plt.show()
print(f"Gráfico salvo como '{nome_grafico}'")

# --- Acumular resultados na tabela ---
todos_resultados.append({
    'Instância': instancia_atual,
    'PLI Custo': f"{pli_cost:.2f}" if pli_cost else "N/A",
    'PLI Tempo (s)': f"{pli_time:.4f}" if pli_cost else "N/A",
    'GA Melhor': f"{ga_best:.2f}",
    'GA Média ± Desvio': f"{ga_mean:.2f} ± {ga_std:.2f}",
    'GA Tempo Médio (s)': f"{ga_time_mean:.4f} ± {ga_time_std:.4f}"
})

# Exibir tabela atualizada
df_final = pd.DataFrame(todos_resultados)
print("\n" + "="*80)
print("TABELA COMPARATIVA GERAL (com desvio padrão)")
print("="*80)
display(df_final)
df_final.to_csv('tabela_comparacao_final.csv', index=False)
print("\nTabela salva em 'tabela_comparacao_final.csv'")
print("-> Para testar outra instância, execute o Bloco 1 novamente.")

# Bloco 5: Análise de Sensibilidade do GA:

In [ ]:
# @title 5. Análise de Sensibilidade do GA (variação do tamanho da população)
print("\n" + "="*80)
print("ANÁLISE DE SENSIBILIDADE - Tamanho da População")
print("="*80)

# Parâmetros fixos
generations = 100
mutation_rate = 0.1
repetitions = 5

# Valores a testar
pop_sizes = [20, 50, 100]
sens_results = []

for pop in pop_sizes:
    print(f"\n--- Testando pop_size = {pop} ---")
    histories, ga_costs, ga_times = run_ga(
        df_nurses, df_rooms, weights,
        pop_size=pop, generations=generations,
        mutation_rate=mutation_rate, repetitions=repetitions
    )
    sens_results.append({
        'pop_size': pop,
        'Melhor': np.min(ga_costs),
        'Média': np.mean(ga_costs),
        'Desvio': np.std(ga_costs),
        'Tempo médio (s)': np.mean(ga_times)
    })

# Exibir tabela de sensibilidade
df_sens = pd.DataFrame(sens_results)
print("\nTabela de Sensibilidade:")
display(df_sens)

# Gráfico comparativo
plt.figure(figsize=(8, 4))
plt.bar(df_sens['pop_size'].astype(str), df_sens['Média'], yerr=df_sens['Desvio'], capsize=5)
plt.xlabel('Tamanho da População')
plt.ylabel('Custo Médio')
plt.title('Sensibilidade do GA ao Tamanho da População')
plt.grid(axis='y')
plt.tight_layout()
plt.show()